In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=7d0e61ca5f5414e08573985d8c0f9c09cb320627c76e2d93a3fd97c416d17856
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wget
pd.options.display.float_format = '{:,.4f}'.format

In [3]:
# Getting the data:

wget.download('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv')

'AB_NYC_2019.csv'

In [4]:
df = pd.read_csv('/content/AB_NYC_2019.csv')
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.6475,-73.9724,Private room,149,1,9,2018-10-19,0.2100,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.7536,-73.9838,Entire home/apt,225,1,45,2019-05-21,0.3800,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.8090,-73.9419,Private room,150,3,0,NaN,nan,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.6851,-73.9598,Entire home/apt,89,1,270,2019-07-05,4.6400,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.7985,-73.9440,Entire home/apt,80,10,9,2018-11-19,0.1000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.6785,-73.9500,Private room,70,2,0,NaN,nan,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.7018,-73.9332,Private room,40,4,0,NaN,nan,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.8147,-73.9487,Entire home/apt,115,10,0,NaN,nan,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.7575,-73.9911,Shared room,55,1,0,NaN,nan,6,2


In [5]:
# Fixing df's text

df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [6]:
df = df.fillna(0)

In [7]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,"48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000","48,895.0000"
mean,"19,017,143.2362","67,620,010.6466",40.7289,-73.9522,152.7207,7.0300,23.2745,1.0909,7.1440,112.7813
std,"10,983,108.3856","78,610,967.0327",0.0545,0.0462,240.1542,20.5105,44.5506,1.5973,32.9525,131.6223
min,"2,539.0000","2,438.0000",40.4998,-74.2444,0.0000,1.0000,0.0000,0.0000,1.0000,0.0000
25%,"9,471,945.0000","7,822,033.0000",40.6901,-73.9831,69.0000,1.0000,1.0000,0.0400,1.0000,0.0000
50%,"19,677,284.0000","30,793,816.0000",40.7231,-73.9557,106.0000,3.0000,5.0000,0.3700,1.0000,45.0000
75%,"29,152,178.5000","107,434,423.0000",40.7631,-73.9363,175.0000,5.0000,24.0000,1.5800,2.0000,227.0000
max,"36,487,245.0000","274,321,313.0000",40.9131,-73.7130,"10,000.0000","1,250.0000",629.0000,58.5000,327.0000,365.0000


In [8]:
df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

In [9]:
features = ['neighbourhood_group',
            'room_type',
            'latitude',
            'longitude',
            'price',
            'minimum_nights',
            'number_of_reviews',
            'reviews_per_month',
            'calculated_host_listings_count',
            'availability_365']

In [10]:
features_r = ['neighbourhood_group',
            'room_type',
            'latitude',
            'longitude',
            'minimum_nights',
            'number_of_reviews',
            'reviews_per_month',
            'calculated_host_listings_count',
            'availability_365']

In [11]:
df[features].isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

Question 1:

What is the most frequent observation (mode) for the column 'neighbourhood_group'?

In [12]:
df.neighbourhood_group.describe()

count         48895
unique            5
top       manhattan
freq          21661
Name: neighbourhood_group, dtype: object

In [13]:
df.groupby(by = 'neighbourhood_group', axis = 0).count()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
neighbourhood_group,,,,,,,,,,,,,,,
bronx,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091,1091
brooklyn,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104,20104
manhattan,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661,21661
queens,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666,5666
staten_island,373,373,373,373,373,373,373,373,373,373,373,373,373,373,373


Split the data:

Split your data in train/val/test sets, with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value ('price') is not in your dataframe.

In [14]:
import sklearn
import sklearn.model_selection
from sklearn.model_selection import train_test_split

In [15]:
df_train_full, df_test = train_test_split(df, test_size = 0.2, random_state=42)

In [16]:
len(df_train_full), len(df_test)

(39116, 9779)

In [17]:
df_train, df_val = train_test_split(df_train_full, test_size = 0.25, random_state=42)

In [18]:
len(df_train), len(df_val)

(29337, 9779)

In [19]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price
y_val = df_val.price
y_test = df_test.price

df_train = df_train[features_r]
df_val = df_val[features_r]
df_test = df_test[features_r]

Question 2:

Create the correlation matrix for the numerical features of your train dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

In [20]:
train_corr = df_train.corr()
train_corr

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.0000,0.0803,0.0274,-0.0062,-0.0072,0.0194,-0.0059
longitude,0.0803,1.0000,-0.0607,0.0551,0.1346,-0.1170,0.0837
minimum_nights,0.0274,-0.0607,1.0000,-0.0760,-0.1207,0.1186,0.1389
number_of_reviews,-0.0062,0.0551,-0.0760,1.0000,0.5904,-0.0732,0.1745
reviews_per_month,-0.0072,0.1346,-0.1207,0.5904,1.0000,-0.0488,0.1654
calculated_host_listings_count,0.0194,-0.1170,0.1186,-0.0732,-0.0488,1.0000,0.2259
availability_365,-0.0059,0.0837,0.1389,0.1745,0.1654,0.2259,1.0000


In [21]:
features_r_num = train_corr.columns
features_r_num

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [22]:
for i in features_r_num:
  print(i, sorted(train_corr[i].to_list())[-2])

latitude 0.08030088258320403
longitude 0.13464215986454384
minimum_nights 0.13890125161845288
number_of_reviews 0.5903739015971261
reviews_per_month 0.5903739015971261
calculated_host_listings_count 0.22591308547640782
availability_365 0.22591308547640782


Question 3:

Calculate the mutual information score for the two categorical variables that we have. Use the training set only.

Which of these two variables has bigger score?
Round it to 2 decimal digits using round(score, 2)

In [23]:
features_r_cat = ['neighbourhood_group', 'room_type']
pd.DataFrame(df_train[features_r_cat].value_counts())

0
neighbourhood_group room_type            
manhattan           entire_home/apt  7980
brooklyn            private_room     6083
                    entire_home/apt  5748
manhattan           private_room     4765
queens              private_room     2025
                    entire_home/apt  1226
bronx               private_room      376
manhattan           shared_room       286
brooklyn            shared_room       247
bronx               entire_home/apt   231
queens              shared_room       113
staten_island       private_room      111
                    entire_home/apt   101
bronx               shared_room        39
staten_island       shared_room         6

In [24]:
pd.DataFrame(df_train[['room_type','neighbourhood_group']].groupby(by='neighbourhood_group').count())

,room_type
neighbourhood_group,
bronx,646
brooklyn,12078
manhattan,13031
queens,3364
staten_island,218


In [25]:
pd.DataFrame(df_train[['room_type','neighbourhood_group']].groupby(by='room_type').count())

,neighbourhood_group
room_type,
entire_home/apt,15286
private_room,13360
shared_room,691


In [26]:
from sklearn.metrics import mutual_info_score

In [27]:
mutual_info_score(y_train, df_train['neighbourhood_group'])

0.11007827465599773

In [28]:
mutual_info_score(y_train, df_train['room_type'])

0.3147677750139868

Question 4:

Now let's train a logistic regression. For that, we need to turn our price prediction problem into a binary classification task.

Let's create a variable above_average which is 1 if the price is above (or equal to) 152.
Remember that we have two categorical variables in the data. Include them using one-hot encoding.
Fit the model on the training dataset.

To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:

model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)

Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [29]:
y_train_2 = pd.DataFrame(y_train)
y_train_2['bin_price'] = [0 if x < 152 else 1 for x in y_train_2['price']] 
y_train_2 = y_train_2.drop(columns='price', axis = 1)
y_train_2 = y_train_2.squeeze(axis=1)
y_train_2

0        0
1        0
2        0
3        0
4        0
        ..
29332    0
29333    0
29334    1
29335    0
29336    0
Name: bin_price, Length: 29337, dtype: int64

In [30]:
y_val_2 = pd.DataFrame(y_val)
y_val_2['bin_price'] = [0 if x < 152 else 1 for x in y_val_2['price']] 
y_val_2 = y_val_2.drop(columns='price', axis = 1)
y_val_2 = y_val_2.squeeze(axis=1)
y_val_2

0       0
1       0
2       1
3       0
4       1
       ..
9774    1
9775    0
9776    0
9777    0
9778    0
Name: bin_price, Length: 9779, dtype: int64

In [31]:
from sklearn.feature_extraction import DictVectorizer

In [32]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [33]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=bronx',
 'neighbourhood_group=brooklyn',
 'neighbourhood_group=manhattan',
 'neighbourhood_group=queens',
 'neighbourhood_group=staten_island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=entire_home/apt',
 'room_type=private_room',
 'room_type=shared_room']

In [34]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,50.0000,13.0000,40.7276,-73.9450,3.0000,0.0000,1.0000,0.0000,0.0000,0.0000,29.0000,0.7000,1.0000,0.0000,0.0000
1,7.0000,1.0000,40.7085,-74.0050,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
2,0.0000,1.0000,40.8315,-73.9277,40.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
3,0.0000,1.0000,40.6645,-73.9941,2.0000,0.0000,1.0000,0.0000,0.0000,0.0000,3.0000,0.0800,1.0000,0.0000,0.0000
4,67.0000,2.0000,40.7412,-74.0001,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,48.0000,1.8000,0.0000,1.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29332,0.0000,1.0000,40.7175,-73.9569,6.0000,0.0000,1.0000,0.0000,0.0000,0.0000,5.0000,0.1300,0.0000,1.0000,0.0000
29333,0.0000,2.0000,40.6640,-73.9854,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,7.0000,0.1700,0.0000,1.0000,0.0000
29334,88.0000,1.0000,40.7999,-73.9700,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,1.0000,0.6400,0.0000,1.0000,0.0000
29335,0.0000,1.0000,40.6959,-73.9634,60.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000


In [35]:
pd.DataFrame(X_val)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,52.0000,1.0000,40.7024,-73.9293,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,35.0000,1.8000,0.0000,1.0000,0.0000
1,343.0000,2.0000,40.6850,-73.9662,14.0000,0.0000,1.0000,0.0000,0.0000,0.0000,4.0000,0.1100,1.0000,0.0000,0.0000
2,260.0000,1.0000,40.6691,-73.9482,3.0000,0.0000,1.0000,0.0000,0.0000,0.0000,153.0000,2.6400,1.0000,0.0000,0.0000
3,0.0000,3.0000,40.7977,-73.9611,3.0000,0.0000,0.0000,1.0000,0.0000,0.0000,2.0000,0.0200,0.0000,1.0000,0.0000
4,365.0000,18.0000,40.7608,-73.9989,30.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,5.0000,1.0000,40.7597,-73.9892,4.0000,0.0000,0.0000,1.0000,0.0000,0.0000,2.0000,0.1400,1.0000,0.0000,0.0000
9775,83.0000,7.0000,40.8095,-73.9300,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,17.0000,1.7900,0.0000,0.0000,1.0000
9776,0.0000,1.0000,40.7306,-73.9829,1.0000,0.0000,0.0000,1.0000,0.0000,0.0000,4.0000,0.1000,0.0000,1.0000,0.0000
9777,0.0000,1.0000,40.6886,-73.9539,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000


In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
X_train

array([[ 50.     ,  13.     ,  40.7276 , ...,   1.     ,   0.     ,
          0.     ],
       [  7.     ,   1.     ,  40.70847, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.83149, ...,   1.     ,   0.     ,
          0.     ],
       ...,
       [ 88.     ,   1.     ,  40.79994, ...,   0.     ,   1.     ,
          0.     ],
       [  0.     ,   1.     ,  40.69585, ...,   0.     ,   1.     ,
          0.     ],
       [281.     ,   2.     ,  40.64438, ...,   1.     ,   0.     ,
          0.     ]])

In [38]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train, y_train_2)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
model.coef_[0].round(5)

array([ 0.00293,  0.00394, -0.22902, -0.09554, -0.01248, -0.39922,
        0.08087,  1.24655, -0.79886, -0.13198, -0.00326, -0.0421 ,
        1.63005, -1.1585 , -0.47418])

In [40]:
model.intercept_.round(5)

array([-0.00264])

In [41]:
model.predict(X_train)

array([0, 0, 0, ..., 0, 0, 1])

In [42]:
model.predict_proba(X_train)

array([[0.62672509, 0.37327491],
       [0.89615208, 0.10384792],
       [0.82567537, 0.17432463],
       ...,
       [0.87785885, 0.12214115],
       [0.98335562, 0.01664438],
       [0.46505707, 0.53494293]])

In [43]:
y_pred = model.predict_proba(X_val)[:,1]
y_pred

array([0.03277337, 0.56905405, 0.39587862, ..., 0.09973834, 0.03416778,
       0.64058828])

In [44]:
prediction_val = (y_pred >= 0.5)
df_val[prediction_val]

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
1,brooklyn,entire_home/apt,40.6850,-73.9662,14,4,0.1100,2,343
4,manhattan,entire_home/apt,40.7608,-73.9989,30,0,0.0000,18,365
6,manhattan,entire_home/apt,40.7324,-74.0093,4,1,0.0300,1,0
7,manhattan,entire_home/apt,40.8063,-73.9627,7,2,0.0500,1,0
9,manhattan,entire_home/apt,40.8086,-73.9457,3,123,1.7300,3,248
...,...,...,...,...,...,...,...,...,...
9763,manhattan,entire_home/apt,40.7353,-73.9807,14,0,0.0000,1,0
9768,manhattan,entire_home/apt,40.8018,-73.9571,8,9,0.1800,1,111
9770,manhattan,entire_home/apt,40.7383,-73.9894,2,2,0.7000,1,166
9774,manhattan,entire_home/apt,40.7597,-73.9892,4,2,0.1400,1,5


In [45]:
np.array((y_val > 151).astype(int))

array([0, 0, 1, ..., 0, 0, 0])

In [46]:
(y_pred >= 0.5).astype(int)

array([0, 1, 0, ..., 0, 0, 1])

In [47]:
np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)

array([ True, False, False, ...,  True,  True, False])

In [48]:
(np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)).mean()

0.7864812353001329

In [49]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = (y_pred >= 0.5).astype(int)
df_pred['actual'] = y_val_2
df_pred['correct'] = (df_pred['prediction'] == df_pred['actual'])
df_pred

,probability,prediction,actual,correct
0,0.0328,0,0,True
1,0.5691,1,0,False
2,0.3959,0,1,False
3,0.0975,0,0,True
4,0.7982,1,1,True
...,...,...,...,...
9774,0.6375,1,1,True
9775,0.0460,0,0,True
9776,0.0997,0,0,True
9777,0.0342,0,0,True


In [50]:
df_pred['correct'].value_counts()

True     7691
False    2088
Name: correct, dtype: int64

In [51]:
df_pred['correct'].mean()

0.7864812353001329

In [52]:
round(df_pred['correct'].mean(), 2)

0.79

Question 5:


We have 9 features: 7 numerical features and 2 categorical.
Let's find the least useful one using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).

Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

1. neighbourhood_group

2. room_type

3. number_of_reviews

4. reviews_per_month

note: the difference doesn't have to be positive

In [53]:
dv.get_feature_names()

['availability_365',
 'calculated_host_listings_count',
 'latitude',
 'longitude',
 'minimum_nights',
 'neighbourhood_group=bronx',
 'neighbourhood_group=brooklyn',
 'neighbourhood_group=manhattan',
 'neighbourhood_group=queens',
 'neighbourhood_group=staten_island',
 'number_of_reviews',
 'reviews_per_month',
 'room_type=entire_home/apt',
 'room_type=private_room',
 'room_type=shared_room']

In [54]:
X_train_no_neighbourhood = pd.DataFrame(X_train)
X_train_no_neighbourhood.columns = dv.get_feature_names()
X_train_no_neighbourhood = X_train_no_neighbourhood.drop(columns = ['neighbourhood_group=bronx',
                                                                    'neighbourhood_group=brooklyn',
                                                                    'neighbourhood_group=manhattan',
                                                                    'neighbourhood_group=queens',
                                                                    'neighbourhood_group=staten_island'])
X_train_no_neighbourhood

,availability_365,calculated_host_listings_count,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,room_type=entire_home/apt,room_type=private_room,room_type=shared_room
0,50.0000,13.0000,40.7276,-73.9450,3.0000,29.0000,0.7000,1.0000,0.0000,0.0000
1,7.0000,1.0000,40.7085,-74.0050,1.0000,0.0000,0.0000,0.0000,1.0000,0.0000
2,0.0000,1.0000,40.8315,-73.9277,40.0000,0.0000,0.0000,1.0000,0.0000,0.0000
3,0.0000,1.0000,40.6645,-73.9941,2.0000,3.0000,0.0800,1.0000,0.0000,0.0000
4,67.0000,2.0000,40.7412,-74.0001,1.0000,48.0000,1.8000,0.0000,1.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...
29332,0.0000,1.0000,40.7175,-73.9569,6.0000,5.0000,0.1300,0.0000,1.0000,0.0000
29333,0.0000,2.0000,40.6640,-73.9854,1.0000,7.0000,0.1700,0.0000,1.0000,0.0000
29334,88.0000,1.0000,40.7999,-73.9700,1.0000,1.0000,0.6400,0.0000,1.0000,0.0000
29335,0.0000,1.0000,40.6959,-73.9634,60.0000,0.0000,0.0000,0.0000,1.0000,0.0000


In [76]:
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_no_neighbourhood, y_train_2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [77]:
X_val_no_neighbourhood = pd.DataFrame(X_val)
X_val_no_neighbourhood.columns = dv.get_feature_names()
X_val_no_neighbourhood = X_val_no_neighbourhood.drop(columns = ['neighbourhood_group=bronx',
                                                                    'neighbourhood_group=brooklyn',
                                                                    'neighbourhood_group=manhattan',
                                                                    'neighbourhood_group=queens',
                                                                    'neighbourhood_group=staten_island'])

In [78]:
y_pred = model.predict_proba(X_val_no_neighbourhood)[:,1]
y_pred

array([0.04868643, 0.67797039, 0.48406693, ..., 0.05328779, 0.05456754,
       0.49418483])

In [79]:
no_neigh_acc = (np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)).mean()
no_neigh_acc

0.7509970344616014

In [68]:
X_train_no_room = pd.DataFrame(X_train)
X_train_no_room.columns = dv.get_feature_names()
X_train_no_room = X_train_no_room.drop(columns = ['room_type=entire_home/apt',
                                                  'room_type=private_room',
                                                  'room_type=shared_room'])
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_no_room, y_train_2)
X_val_no_room = pd.DataFrame(X_val)
X_val_no_room.columns = dv.get_feature_names()
X_val_no_room = X_val_no_room.drop(columns = ['room_type=entire_home/apt',
                                              'room_type=private_room',
                                              'room_type=shared_room'])
y_pred = model.predict_proba(X_val_no_room)[:,1]
no_room_acc = (np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)).mean()
no_room_acc

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7165354330708661

In [69]:
X_train_no_rev = pd.DataFrame(X_train)
X_train_no_rev.columns = dv.get_feature_names()
X_train_no_rev = X_train_no_rev.drop(columns = ['number_of_reviews'])
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_no_rev, y_train_2)
X_val_no_rev = pd.DataFrame(X_val)
X_val_no_rev.columns = dv.get_feature_names()
X_val_no_rev = X_val_no_rev.drop(columns = ['number_of_reviews'])
y_pred = model.predict_proba(X_val_no_rev)[:,1]
no_rev_num_acc = (np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)).mean()
no_rev_num_acc

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7869925350240311

In [71]:
X_train_no_rev_month = pd.DataFrame(X_train)
X_train_no_rev_month.columns = dv.get_feature_names()
X_train_no_rev_month = X_train_no_rev_month.drop(columns = ['reviews_per_month'])
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(X_train_no_rev_month, y_train_2)
X_val_no_rev_month = pd.DataFrame(X_val)
X_val_no_rev_month.columns = dv.get_feature_names()
X_val_no_rev_month = X_val_no_rev_month.drop(columns = ['reviews_per_month'])
y_pred = model.predict_proba(X_val_no_rev_month)[:,1]
no_rev_month_acc = (np.array((y_val > 151).astype(int)) == (y_pred >= 0.5).astype(int)).mean()
no_rev_month_acc

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7850495960732181

In [80]:
no_neigh_acc, no_rev_month_acc, no_rev_num_acc, no_room_acc

(0.7509970344616014,
 0.7850495960732181,
 0.7869925350240311,
 0.7165354330708661)

Question 6:

For this question, we'll see how to use a linear regression model from Scikit-Learn. We'll need to use the original column 'price'. 

Apply the logarithmic transformation to this column. Fit the Ridge regression model on the training data. This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]

Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
If there are multiple options, select the smallest alpha.

In [96]:
df['log_price'] = np.log1p(df.price)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

In [93]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [88]:
alpha = [0, 0.01, 0.1, 1, 10]

In [106]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0)
model.fit(X_train, y_train_log)
y_pred_val_log = model.predict(X_val)
rmse_0 = rmse(y_val_log, y_pred_val_log)
rmse_0

0.4971901903517499

In [107]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.01)
model.fit(X_train, y_train_log)
y_pred_val_log = model.predict(X_val)
rmse_0_01 = rmse(y_val_log, y_pred_val_log)
rmse_0_01

0.4971173046190628

In [108]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.1)
model.fit(X_train, y_train_log)
y_pred_val_log = model.predict(X_val)
rmse_0_1 = rmse(y_val_log, y_pred_val_log)
rmse_0_1

0.4971183244694407

In [109]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=1)
model.fit(X_train, y_train_log)
y_pred_val_log = model.predict(X_val)
rmse_1 = rmse(y_val_log, y_pred_val_log)
rmse_1

0.4971395363320047

In [110]:
from sklearn.linear_model import Ridge
model = Ridge(alpha=10)
model.fit(X_train, y_train_log)
y_pred_val_log = model.predict(X_val)
rmse_10 = rmse(y_val_log, y_pred_val_log)
rmse_10

0.49788660158765524

In [111]:
rmse_0, rmse_0_01, rmse_0_1, rmse_1, rmse_10

(0.4971901903517499,
 0.4971173046190628,
 0.4971183244694407,
 0.4971395363320047,
 0.49788660158765524)

In [112]:
min(rmse_0, rmse_0_01, rmse_0_1, rmse_1, rmse_10)

0.4971173046190628

In [55]:
from hashlib import sha1

def compute_hash(email):
    return sha1(email.lower().encode('utf-8')).hexdigest()

In [56]:
compute_hash("dezh.econ@gmail.com")

'820ecc29bc0c94cfd3fc925eedfeba5f04e7b8dc'